In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Install required packages
!pip install torch torchcrf transformers tqdm pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
# Set up paths (adjust these paths based on your actual file structure in Colab)
import os
import sys
import torch
import pandas as pd
from transformers import XLMRobertaTokenizer
from tqdm import tqdm

In [3]:
!pip install pytorch-crf

In [4]:
# Add the project root to Python path for imports
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/nlp_project_roberta"  # Change this if your files are in a different location
sys.path.append(PROJECT_DIR)

# Import the necessary modules
from dataset.dataset import get_dataloaders
from Models.xlm_roberta_improved import XLMRobertaForIdiomDetection, train_model, predict_idioms, evaluate

# Define paths
DATA_DIR = os.path.join(PROJECT_DIR, "dataset")
MODELS_DIR = os.path.join(PROJECT_DIR, "Models")
OUTPUT_DIR = os.path.join(PROJECT_DIR, "Submissions")

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
# Function to train the model
def run_training(epochs=10, batch_size=8, lr=5e-6, max_length=128):
    # Initialize XLM-RoBERTa tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    print("Loading data...")
    # Load and preprocess data
    train_loader, val_loader, _ = get_dataloaders(
        train_path=os.path.join(DATA_DIR, 'train.csv'),
        val_path=os.path.join(DATA_DIR, 'eval.csv'),
        batch_size=batch_size,  # Reduced batch size
        max_length=max_length,
        tokenizer=tokenizer
    )

    print("Starting training...")
    # Train the model with modified parameters
    model = train_model(
        train_loader, val_loader, tokenizer,
        epochs=epochs, lr=lr  # Lower learning rate
    )

    print('Training complete. Best model saved as best_xlm_roberta_idiom_model.pt')
    return model


In [6]:
# Function to evaluate the model
def run_evaluation(batch_size=16, max_length=128):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize XLM-RoBERTa tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    # Load data
    print("Loading evaluation data...")
    _, val_loader, _ = get_dataloaders(
        train_path=os.path.join(DATA_DIR, 'train.csv'),
        val_path=os.path.join(DATA_DIR, 'eval.csv'),
        batch_size=batch_size,
        max_length=max_length,
        tokenizer=tokenizer
    )

    # Load the model
    print("Loading model...")
    model = XLMRobertaForIdiomDetection()
    model.load_state_dict(torch.load('best_xlm_roberta_idiom_model.pt', map_location=device))
    model.to(device)

    # Evaluate the model
    print("Evaluating model...")
    metrics = evaluate(model, val_loader, tokenizer, device)
    print('Evaluation complete.')
    print(metrics)

    return metrics

In [7]:
# Function to generate predictions
def run_prediction(batch_size=16, max_length=128, output_file='prediction_xlm.csv'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize XLM-RoBERTa tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    # Load the model
    print("Loading model...")
    model = XLMRobertaForIdiomDetection()
    model.load_state_dict(torch.load('best_xlm_roberta_idiom_model.pt', map_location=device))
    model.to(device)

    # Read test data
    print("Reading test data...")
    test_df = pd.read_csv(os.path.join(DATA_DIR, 'eval_w_o_labels.csv'))
    ids = test_df['id'].tolist()
    languages = test_df['language'].tolist()
    sentences = test_df['sentence'].tolist()

    # Generate predictions
    print("Generating predictions...")
    results = []
    for idx, sentence, lang in tqdm(zip(ids, sentences, languages), total=len(ids)):
        _, idiom_indices = predict_idioms(model, tokenizer, sentence, device)
        # If no idiom, output [-1] as in training
        if not idiom_indices:
            idiom_indices = [-1]
        results.append({'id': idx, 'indices': str(idiom_indices), 'language': lang})

    # Create output dataframe and save to file
    output_path = os.path.join(OUTPUT_DIR, output_file)
    out_df = pd.DataFrame(results)
    out_df.to_csv(output_path, index=False)
    print(f'Predictions saved to {output_path}')

    return out_df

In [8]:
# Train the model (this will take time)
model = run_training(epochs=10, batch_size=16, lr=2e-5)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

Validation loss: 1.8275
F1 Score: 1.3156
No improvement for 2 epochs


Epoch 6/10 [Training]: 100%|██████████| 720/720 [03:05<00:00,  3.87it/s, loss=0.4956]


Average training loss: 1.0358


Evaluating: 100%|██████████| 90/90 [00:23<00:00,  3.84it/s]

Validation loss: 2.3398
F1 Score: 1.3372
No improvement for 3 epochs
Early stopping triggered after 6 epochs
Training complete. Best model saved as best_xlm_roberta_idiom_model.pt


In [9]:

# Evaluate the model
metrics = run_evaluation()

Loading evaluation data...
Loading model...
Evaluating model...


Evaluating: 100%|██████████| 90/90 [00:23<00:00,  3.80it/s]

Evaluation complete.
{'val_loss': 1.7630106197463142, 'precision': 1.3522050059594755, 'recall': 1.3315727699530517, 'f1': 1.3418095801301004}


In [10]:
# Generate predictions
predictions = run_prediction(output_file='prediction_xlm.csv')

Loading model...
Reading test data...
Generating predictions...


100%|██████████| 1437/1437 [00:44<00:00, 32.32it/s]

Predictions saved to /content/drive/MyDrive/Colab Notebooks/nlp_project_roberta/Submissions/prediction_xlm.csv


In [11]:
# Optional: Test the model with a single example
def test_single_example():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

    model = XLMRobertaForIdiomDetection()
    model.load_state_dict(torch.load('best_xlm_roberta_idiom_model.pt', map_location=device))
    model.to(device)

    # Example sentence from the dataset
    sentence = "Zaman öldürmek, bir tür cinayettir, Zamanınızı öldürmeyin; zamanınızı doldurun, Kıymetini bilerek zamanınızı değerlendirin."
    processed_sentence, idiom_indices = predict_idioms(model, tokenizer, sentence, device)

    print("Sentence:", sentence)
    print("Detected Idiom Indices:", idiom_indices)

    if idiom_indices:
        words = sentence.split()
        try:
            idiom_words = [words[idx] for idx in idiom_indices]
            print("Idiom Words:", " ".join(idiom_words))
        except IndexError:
            print("Warning: Index out of range. Check tokenization.")
    else:
        print("No idiom detected.")

# Uncomment to test with a single example
test_single_example()

Sentence: Zaman öldürmek, bir tür cinayettir, Zamanınızı öldürmeyin; zamanınızı doldurun, Kıymetini bilerek zamanınızı değerlendirin.
Detected Idiom Indices: [0, 1]
Idiom Words: Zaman öldürmek,
